<a href="https://colab.research.google.com/github/danb-neo4j/patient_journey_chatGPT/blob/main/patientJourney_dataLoad_chatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neo4j GDS Patient Journey Demo: Data Loading with ChatGPT
This notebook is an updated version of the original [patient journey data loading notebook](https://github.com/danb-neo4j/patient_journey/blob/main/patientJourney_dataLoad.ipynb). As with the updated [tabular EDA notebook](https://github.com/danb-neo4j/patient_journey_chatGPT/blob/main/patientJourney_tabularEDA_chatGPT.ipynb), this one explores when and how to incorporate ChatGPT into the process of developing graph data models and ingesting tabular data into Neo4j. 

This notebook uses the same [Synthea](https://synthea.mitre.org/) data data sets as before:
* Patients
* Encounters
* Procedures
* Medications (Drugs)

The existing Cypher data loading code code below is adapted from the code provided in [Graph Data Processing with Cypher](https://github.com/PacktPublishing/Cypher-Querying). 

Also as before, this data model **does not** incorporate temporal `HAS_NEXT` types of relationships. Creating such relationships can be very beneficial for patient or customer journey analysis. However, it is also an advanced data modeling topic and can require substantial refactoring of an existing data model. 

The data model used as the basis for this notebook may be more intuitive and represenative of existing data models. The subsequent analysis notebooks also demonstrate how it is possible to identify high-value patient journey insights without incorporating the temporal relationships. Organizations can find the path to generating valuable, graph-based insights quicker by starting with their data as-is before refactoring it for temporal journey analysis. 

*Last updated: 11 May 2023*

## Comments About ChatGPT
I found the process of using ChatGPT to create this data loading notebook quite different than when I was creating the [tabular EDA notebook](https://github.com/danb-neo4j/patient_journey_chatGPT/blob/main/patientJourney_tabularEDA_chatGPT.ipynb). In particular:
* The tabular EDA notebook mostly required me to ask simple, straightforward questions to ChatGPT. Other than creating a few lines of Python code, ChatGPT was just answering questions and interpreting simple data distributions. There were not situations where I asked it complex questions or to act in a generative manner.
* In this notebook I asked ChatGPT to be generative with much more complex prompts. I could see my questions for generating Cypher become much more complex as I worked through the notebook, in part because my requriements were also more complex. As this process continued, ChatGPT seemed to struggle more and more with its responses. 

Working through this notebook was also educational for me in other ways:
* LLM Models are Important: This notebook uses ChatGPT 3.5 Turbo via the Python API. As I was developing the notebook I asked similar questions to ChatGPT 4.0 via the OpenAI web interface. I found the responses from 4.0 on the web interface to, in general, be much better than those I received from 3.5 via the Python API. 
* Prompts are Important: Specific models aside, the quality of the prompt has a significant impact on the quality of the response. I cannot remember the source, but one of the best suggestions I heard for writing prompts was to think of the LLM as a very smart high school aged intern who needs detailed, step-by-step guidance to work from. 
* Graph Data Models are Difficult: There is an art and science to building graph data models and they often depend on many other factors in addition to the data itself. That makes this task for LLMs difficult as well. As with everything related to LLMs, generating Cypher and graph data models is new and will improve as we improve our prompts and the models improve themselves. 

# Import Libraries

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

import matplotlib.pyplot as plt 
import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette("colorblind")
sns.set(rc={'figure.figsize':(12,7)})

import os
import configparser
from IPython.display import Image

In [2]:
# install or import Neo4j GraphDataScience library
try: 
  from graphdatascience import GraphDataScience
  print('Successfully imported GraphDataScience')
except ModuleNotFoundError:
  !pip install graphdatascience
  from graphdatascience import GraphDataScience
  print('installed and imported GraphDataScience')

Successfully imported GraphDataScience


## Mount Google Drive and Set Directory

In [3]:
from google.colab import drive 
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# change working directory 
os.chdir('drive/MyDrive/Colab Notebooks/gds_demos/patient_journey/')
DATA_DIRECTORY = 'data/'

# OpenAI ChatGPT

## Install Python Library

In [5]:
# import or install openai
try: 
    import openai
    print('Successfully imported openai')
except ModuleNotFoundError:
    !pip install openai
    import openai 
    print('installed and imported GraphDataScience')

Successfully imported openai


## Set API Key

In [6]:
# assign openai api key
api_key_path = 'openAI/openAI_API.txt'
openai.api_key = open(api_key_path, 'r').read().strip('\n')

## Set Model and Test Connection

In [11]:
# set openai model 
model = 'gpt-3.5-turbo'
message_history = []

In [12]:
# test api connection
prompt='How many moons does the planet Jupiter have?'
messages = [{'role':'user', 'content': prompt}]

response = openai.ChatCompletion.create(model=model, messages=messages, temperature=0)

In [13]:
response.choices[0].message['content']

'Jupiter has 79 known moons.'

## Helper Function to Capture ChatGPT Message History

In [14]:
def submit_query_and_append(prompt=prompt, message_history=message_history):
    """Submit a query to the openai chatbot and append the response to the message history"""
    # append the prompt to the message history
    message_history.append({'role':'user', 'content': prompt})

    # submit query to openai chatbot
    response = openai.ChatCompletion.create(model=model, messages=message_history, temperature=0)

    # append the response to the message history 
    response_text = response.choices[0].message['content']
    message_history.append({'role':'assistant', 'content': response_text})

    return message_history, response_text

# Connect to Neo4j Database

In [15]:
# import utility function to connect to Neo4j DB
from neoUtils import read_neo4j_properties

In [16]:
# read in Neo4j host and authentication 
NEO4J_PROPERTIES_FILE = 'auth/patientJourney_auth.ini'
HOST, USERNAME, PASSWORD = read_neo4j_properties(NEO4J_PROPERTIES_FILE=NEO4J_PROPERTIES_FILE)

Using HOST, USERNAME, PASSWORD from .ini file


In [17]:
# connect to and instantiate GDS
gds = GraphDataScience(HOST, auth=(USERNAME, PASSWORD), aura_ds=False)

# confirm connection with gds version 
print('Neo4j GDS Version:', gds.version())

Neo4j GDS Version: 2.3.4+17


In [19]:
# # code to delete database, if necessary
# gds.run_cypher('''
# MATCH (n) 
# CALL { WITH n 
# DETACH DELETE n 
# } IN TRANSACTIONS OF 50000 ROWS
# ''')

""


# Read-In Data

## Load Patient Data

In [20]:
# NOTE: The 'updated' version of the Patient Data includes an 'AGE' feature
patient_df = pd.read_csv(DATA_DIRECTORY + 'patients_updated.csv')
patient_df = patient_df.fillna('')
patient_df.shape

(5885, 14)

In [21]:
# convert ZIP feature to string
patient_df['ZIP'] = patient_df['ZIP'].astype(str)

# pad ZIP feature with zeros to the left
patient_df['ZIP'] = patient_df['ZIP'].str.pad(width=5, side='left', fillchar='0')

## Load Encounters Data

In [22]:
encounters_df = pd.read_csv(DATA_DIRECTORY + 'encounters.csv')
encounters_df = encounters_df.fillna('')
encounters_df.shape

(497415, 15)

## Load Procedures Data

In [23]:
procedures_df = pd.read_csv(DATA_DIRECTORY + 'procedures.csv')
procedures_df = procedures_df.fillna('')
procedures_df.shape

(785604, 9)

## Load Medications (Drugs) Data

In [24]:
medications_df = pd.read_csv(DATA_DIRECTORY + 'medications.csv')
medications_df = medications_df.fillna('')
medications_df.shape

(604103, 13)

# Ask ChatGPT About Overall Patient Journey Data Model
Although we have the rough outline of a data model from [Graph Data Processing with Cypher](https://github.com/PacktPublishing/Cypher-Querying), we want to explore what ideas ChatGPT has and if there are any improvements it could make. 

Using ChatGPT to create graph data models is an emerging area, so we will try different prompts in an attempt to produce a workable solution. We will include context about all four data sets in the prompt below to help it develop a solution. 

In [25]:
prompt = f"""I want you to act as an experienced graph data scientist and help me create a graph data model for a Neo4j database.
             My objective is to perform a graph-based patient journey analysis. 
             My data is in the following four pandas dataframes, details of which are between the four hashes:\n
             ####
             patients_df, which contains patient information. This data has {patient_df.shape[0]} records and the following features and datatypes \n{patient_df.dtypes}\n 
             
             encounters_df, which contains data about medical encounters the patients had. 
             This data has {encounters_df.shape[0]} records and the following features and data types \n{encounters_df.columns}\n
             
             procedures_df, which contains medical procedure information associated with patient encounters.
             This data has {procedures_df.shape[0]} records and the following features and data types \n{procedures_df.columns}\n
             
             medications_df, which contains medication data associated with patient encounters and procedures.
             This data has {medications_df.shape[0]} records and the following features and data types {medications_df.columns}\n 
             ####
             
             I want to create a graph data model that will allow me to perform a patient journey analysis using standard Cypher as well
             as algorithms in the Neo4j Graph Data Science library. 
             Please design a graph data model with recommendations for nodes, properties, and relationships.
"""

message_history = []

message_history, response_text = submit_query_and_append(prompt, message_history)

print(response_text)

Based on the provided data, the following graph data model is recommended:

Nodes:
- Patient: with properties Id, FIRST, LAST, SSN, BIRTHDATE, DEATHDATE, DECEASED, AGE, GENDER, RACE, INCOME, HEALTHCARE_EXPENSES, ZIP
- Encounter: with properties Id, START, STOP, ORGANIZATION, PROVIDER, PAYER, ENCOUNTERCLASS, CODE, DESCRIPTION, BASE_ENCOUNTER_COST, TOTAL_CLAIM_COST, PAYER_COVERAGE, REASONCODE, REASONDESCRIPTION
- Procedure: with properties START, STOP, CODE, DESCRIPTION, BASE_COST, REASONCODE, REASONDESCRIPTION
- Medication: with properties START, STOP, PAYER, CODE, DESCRIPTION, BASE_COST, PAYER_COVERAGE, DISPENSES, TOTALCOST, REASONCODE, REASONDESCRIPTION

Relationships:
- (Patient)-[:HAD_ENCOUNTER]->(Encounter)
- (Encounter)-[:HAD_PROCEDURE]->(Procedure)
- (Encounter)-[:HAD_MEDICATION]->(Medication)

This model allows for easy traversal of patient journeys, as well as analysis of encounters, procedures, and medications associated with each patient. The relationships between nodes also 

In [26]:
prompt = f"""Continuing the conversation. You provided good initial data model. 
             Are there any ways to optimize it, especially for use with Graph Data Science?
             Which node properties are the most important? 
             Which relationships are the most important?
             Are there any node properties that could become independent nodes? Or properties that should be added to relationships?
             Please provide suggestions for how to optimize the data model you suggested.
"""

message_history, response_text = submit_query_and_append(prompt, message_history)

print(response_text)

Yes, there are several ways to optimize the data model for use with Graph Data Science. Here are some suggestions:

1. Add labels to nodes: Adding labels to nodes can help optimize queries by allowing for more efficient indexing and filtering. For example, adding the label ":Patient" to the Patient nodes can help optimize queries that involve only patient data.

2. Use relationship types: Using specific relationship types can help optimize queries by allowing for more efficient traversal of the graph. For example, using the relationship type "HAD_PROCEDURE" instead of a generic "HAS" relationship can help optimize queries that involve procedures.

3. Add indexes: Adding indexes to frequently queried properties can help optimize queries by allowing for faster lookups. For example, adding an index to the "CODE" property in the Procedure and Medication nodes can help optimize queries that involve specific procedures or medications.

4. Use property graphs: Property graphs allow for the cr

In [27]:
prompt = f"""Continuing the conversation. Please provide me with an updated graph data model 
             and include the most impactful optimizations you suggested in your previous response.
"""

message_history, response_text = submit_query_and_append(prompt, message_history)

print(response_text)

Based on the previous suggestions, here is an updated graph data model:

Nodes:
- :Patient with properties Id, FIRST, LAST, SSN, BIRTHDATE, DEATHDATE, DECEASED, AGE, GENDER, RACE, INCOME, HEALTHCARE_EXPENSES
- :Encounter with properties Id, START, STOP, ORGANIZATION, PROVIDER, PAYER, ENCOUNTERCLASS, BASE_ENCOUNTER_COST, TOTAL_CLAIM_COST, PAYER_COVERAGE, REASONCODE, REASONDESCRIPTION
- :Procedure with properties CODE, DESCRIPTION, BASE_COST, REASONCODE, REASONDESCRIPTION
- :Medication with properties CODE, DESCRIPTION, BASE_COST, PAYER_COVERAGE, DISPENSES, TOTALCOST, REASONCODE, REASONDESCRIPTION
- :ZipCode with property ZIP

Relationships:
- (Patient)-[:HAD_ENCOUNTER]->(Encounter)
- (Encounter)-[:HAD_PROCEDURE {COST}]->(Procedure)
- (Encounter)-[:HAD_MEDICATION {COST}]->(Medication)
- (Patient)-[:LOCATED_IN]->(ZipCode)

Optimizations:
1. Labels have been added to nodes to improve query performance.
2. Specific relationship types have been used to improve query performance.
3. Indexes h

# Load Patient Data into Neo4j

## Ask ChatGPT Specific Questions

In [28]:
# view patient dataframe for reference 
patient_df.head()

,Unnamed: 0,Id,FIRST,LAST,SSN,BIRTHDATE,DEATHDATE,DECEASED,AGE,GENDER,RACE,INCOME,HEALTHCARE_EXPENSES,ZIP
0,0,75bef938-819a-ce94-8dc9-17dc50077a86,Joi660,Barrows492,999-68-3502,1991-11-22,,False,31,F,white,144427,36108.19,02720
1,1,faf277d8-c5c7-406c-f139-9143c35ff4e8,Billy698,McLaughlin530,999-51-6719,1987-09-28,,False,35,M,white,10147,6685.55,00000
2,2,4f01341c-4946-31cd-9f2e-7844867f1606,Dominic463,Ward668,999-82-1028,1979-07-30,,False,43,M,white,109151,37041.63,00000
3,3,63cd703f-3077-613f-adf0-85fc32f3ae2a,Ethan766,Nolan344,999-72-1131,1971-01-22,,False,52,M,white,78360,16736.57,01086
4,4,61b942b2-8324-0208-7e2f-5309af2be4f4,Arron144,Bashirian201,999-12-8574,1968-04-06,,False,55,M,asian,416323,57242.58,02210


In [29]:
prompt = f"""
Continuing the conversation. I am going to load the patient nodes into Neo4j from 
the patient_df dataframe I shared earlier. I am creating a Patient node with properties
from the following features:\n
"FIRST, LAST, SSN, GENDER, BIRTHDATE, DEATHDATE, DECEASED, AGE, HEALTHCARE_EXPENSES, INCOME"
\n
Additionally, I plan to create Zip_Code nodes from the "Zip" feature and 
Race codes from the "Race" feature. Both will connect to the Patient node.

Please provide me with Cypher code to load this data into Neo4j. 
Please also provide an explanation for the code to help me understand it.
"""

message_history, response_text = submit_query_and_append(prompt, message_history)

print(response_text)

Sure, here is the Cypher code to load the patient nodes, as well as the Zip_Code and Race nodes:

```
// Create Zip_Code nodes
LOAD CSV WITH HEADERS FROM 'file:///patients.csv' AS row
MERGE (z:Zip_Code {ZIP: row.ZIP})

// Create Race nodes
LOAD CSV WITH HEADERS FROM 'file:///patients.csv' AS row
MERGE (r:Race {code: row.RACE})

// Create Patient nodes and connect to Zip_Code and Race nodes
LOAD CSV WITH HEADERS FROM 'file:///patients.csv' AS row
MERGE (p:Patient {Id: row.Id})
SET p.FIRST = row.FIRST,
    p.LAST = row.LAST,
    p.SSN = row.SSN,
    p.GENDER = row.GENDER,
    p.BIRTHDATE = row.BIRTHDATE,
    p.DEATHDATE = row.DEATHDATE,
    p.DECEASED = row.DECEASED,
    p.AGE = row.AGE,
    p.HEALTHCARE_EXPENSES = row.HEALTHCARE_EXPENSES,
    p.INCOME = row.INCOME
MERGE (p)-[:LOCATED_IN]->(z)
MERGE (p)-[:BELONGS_TO]->(r)
```

Explanation:
- The first section of the code creates Zip_Code nodes by loading the "patients.csv" file and merging a new node for each unique ZIP code found in the

In [31]:
prompt = f"""
Continuing the conversation about loading the patient data, what constraints should I create in the database? 
And what indexes should I create.
Please note that new versions of Neo4j have replaced ASSERT with REQUIRE for creating constraints. 
"""

message_history, response_text = submit_query_and_append(prompt, message_history)

print(response_text)

When creating constraints in Neo4j, it is important to ensure that the constraints are unique and enforceable. Here are some suggestions for constraints and indexes based on the data model and the data being loaded:

Constraints:
- :Patient(Id) REQUIRE: This constraint ensures that each patient node has a unique ID property.
- :Zip_Code(ZIP) REQUIRE: This constraint ensures that each Zip_Code node has a unique ZIP property.
- :Race(code) REQUIRE: This constraint ensures that each Race node has a unique code property.

Indexes:
- :Patient(Id): This index can help optimize queries that involve specific patient IDs.
- :Zip_Code(ZIP): This index can help optimize queries that involve specific ZIP codes.
- :Race(code): This index can help optimize queries that involve specific race codes.

To create these constraints and indexes, you can use the following Cypher code:

```
// Create constraints
CREATE CONSTRAINT ON (p:Patient) ASSERT p.Id IS UNIQUE
CREATE CONSTRAINT ON (z:Zip_Code) ASSERT z

## Create Patient Constraints

In [32]:
# create selected database constraints
gds.run_cypher('''CREATE CONSTRAINT patient_id IF NOT EXISTS FOR (n:Patient) REQUIRE n.id IS UNIQUE''')
gds.run_cypher('''CREATE CONSTRAINT race_type IF NOT EXISTS FOR (n:Race) REQUIRE n.type IS UNIQUE''')
gds.run_cypher('''CREATE CONSTRAINT zip_code IF NOT EXISTS FOR (n:ZipCode) REQUIRE n.code IS UNIQUE''')

""


## Import Patient Data and Relationships

The Cypher above is a good place to start, but as expected is not the final code we would want to use. Additionally, even though we provided context to ChatGPT for the constraints, it still used ASSERT rather than REQUIRE. 

DEATHDATE ended up being excluded due to issues loading missing values as date types. This information is captured via the AGE and DECEASED columns, so we are not losing that context. 

In [33]:
gds.run_cypher('''
CYPHER runtime=slotted
UNWIND $patient_data AS row

CALL {
WITH row
MERGE (p:Patient {id: row.Id})
SET
    p.firstName = row.FIRST,
    p.lastName = row.LAST,
    p.ssn = row.SSN,
    p.gender = row.GENDER,
    p.birthDate = DATE(row.BIRTHDATE),
    p.deceased = toBoolean(row.DECEASED),
    p.age = toFloat(row.AGE),
    p.income = toFloat(row.INCOME),
    p.healthExpenses = toFloat(row.HEALTHCARE_EXPENSES)

WITH row, p
MERGE (r:Race {type: row.RACE})
MERGE (p)-[:HAS_RACE]->(r)

WITH row, p
MERGE (z:ZipCode {code: row.ZIP})
MERGE (p)-[:HAS_ZIP]->(z)

} IN TRANSACTIONS OF 1000 ROWS
''', {'patient_data': patient_df.to_dict('records')})

""


# Load Encounter Data into Neo4j

## Ask ChatGPT Specific Questions

In [34]:
# view encounters dataframe for reference 
encounters_df.head()

,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
0,b7d4cb84-84b8-55a6-186c-fc2a9baa9e12,2010-01-15T11:35:37Z,2010-01-15T12:28:52Z,75bef938-819a-ce94-8dc9-17dc50077a86,fb4a08b8-57ee-382b-b3cb-43d94f1bb4c0,ca383a2c-61e9-319e-8afa-97bc71679361,0133f751-9229-3cfd-815f-b6d4979bdd6a,wellness,162673000,General examination of patient (procedure),136.8,1233.93,1233.93,,
1,896d8330-07e5-9c95-3f71-a9033c13f837,2005-11-21T11:16:28Z,2005-11-21T12:03:47Z,faf277d8-c5c7-406c-f139-9143c35ff4e8,071a898d-3535-39fd-bf61-d33301aacd04,c7efc940-0b81-3a2b-8d26-c3fcad47abcb,df166300-5a78-3502-a46a-832842197811,wellness,162673000,General examination of patient (procedure),136.8,1302.70,1152.70,,
2,4c55d50d-930c-2db6-27e7-2ec206adc81a,2006-11-27T11:16:28Z,2006-11-27T12:13:43Z,faf277d8-c5c7-406c-f139-9143c35ff4e8,071a898d-3535-39fd-bf61-d33301aacd04,c7efc940-0b81-3a2b-8d26-c3fcad47abcb,df166300-5a78-3502-a46a-832842197811,wellness,162673000,General examination of patient (procedure),136.8,704.20,704.20,,
3,fc37d497-489e-fafd-4641-bcab2211611b,2009-11-30T11:16:28Z,2009-11-30T12:04:00Z,faf277d8-c5c7-406c-f139-9143c35ff4e8,071a898d-3535-39fd-bf61-d33301aacd04,c7efc940-0b81-3a2b-8d26-c3fcad47abcb,df166300-5a78-3502-a46a-832842197811,wellness,162673000,General examination of patient (procedure),136.8,1186.78,1086.78,,
4,21f8b4bc-8f2d-18f2-60bc-6e6e2e1edbaf,2012-12-03T11:16:28Z,2012-12-03T11:52:22Z,faf277d8-c5c7-406c-f139-9143c35ff4e8,071a898d-3535-39fd-bf61-d33301aacd04,c7efc940-0b81-3a2b-8d26-c3fcad47abcb,df166300-5a78-3502-a46a-832842197811,wellness,162673000,General examination of patient (procedure),136.8,704.20,604.20,,


In [36]:
# drop the first chats due to token limits
message_history = message_history[5:]

In [35]:
prompt = f"""
Continuing the conversation. I am going to load the encounter nodes into Neo4j from 
the encounters_df dataframe I shared earlier. I am creating an Encounter node with properties
from the following features:\n "START, DESCRIPTION, TOTAL_CLAIM_COST" \n

I also want to include the "STOP" feature as a date property on the Encounter 
node where it is not null or empty in the encounter_df dataframe.

I want to use the "CODE" feature to create a new SNOMED_CT node, with a unique node for each CODE value.
Encounter nodes will connect to SNOMED_CT nodes with an OF_TYPE relationship.
Patient nodes will connect to Encounter nodes with a HAS_ENCOUNTER relationship.

I want to use apoc.create.setLabels to create a secondary label on each Encounter node
based upon the "ENCOUNTERCLASS" feature.

Please provide me with Cypher code to load this data into Neo4j. 
Please also provide an explanation for the code to help me understand it.
"""

message_history, response_text = submit_query_and_append(prompt, message_history)

print(response_text)

Sure, here is the Cypher code to load the encounter nodes, as well as the SNOMED_CT nodes:

```
// Create SNOMED_CT nodes
LOAD CSV WITH HEADERS FROM 'file:///encounters.csv' AS row
MERGE (s:SNOMED_CT {code: row.CODE})

// Create Encounter nodes and connect to SNOMED_CT and Patient nodes
LOAD CSV WITH HEADERS FROM 'file:///encounters.csv' AS row
WITH row
WHERE row.STOP IS NOT NULL AND row.STOP <> ''
MERGE (e:Encounter {Id: row.Id})
SET e.START = row.START,
    e.DESCRIPTION = row.DESCRIPTION,
    e.TOTAL_CLAIM_COST = row.TOTAL_CLAIM_COST
MERGE (e)-[:OF_TYPE]->(s)
MERGE (p:Patient {Id: row.PATIENT})
MERGE (p)-[:HAS_ENCOUNTER]->(e)

// Add secondary label to Encounter nodes based on ENCOUNTERCLASS
CALL apoc.create.setLabels('Encounter', ['Encounter', row.ENCOUNTERCLASS], [e])
```

Explanation:
- The first section of the code creates SNOMED_CT nodes by loading the "encounters.csv" file and merging a new node for each unique SNOMED_CT code found in the file.
- The second section of the code

In [37]:
prompt = f"""
Continuing the conversation. I would like you to update the Cypher code you wrote.
I want to create an Encounter node for all encounters, not just those with a STOP value. 
If an Encounter node is null or empty for STOP, still create the node, 
but do not add the STOP value as a property. 

Also, is there a dynamic way to use apoc.create.setLabels() to dynamically create
secondary Encounter labels as the nodes are created?

Please provide updated Cypher code.
"""

message_history, response_text = submit_query_and_append(prompt, message_history)

print(response_text)

Sure, here is the updated Cypher code to create Encounter nodes for all encounters, and to use apoc.create.setLabels() dynamically:

```
// Create SNOMED_CT nodes
LOAD CSV WITH HEADERS FROM 'file:///encounters.csv' AS row
MERGE (s:SNOMED_CT {code: row.CODE})

// Create Encounter nodes and connect to SNOMED_CT and Patient nodes
LOAD CSV WITH HEADERS FROM 'file:///encounters.csv' AS row
MERGE (e:Encounter {Id: row.Id})
SET e.START = row.START,
    e.DESCRIPTION = row.DESCRIPTION,
    e.TOTAL_CLAIM_COST = row.TOTAL_CLAIM_COST
WITH e, row
WHERE row.STOP IS NOT NULL AND row.STOP <> ''
SET e.STOP = row.STOP
MERGE (e)-[:OF_TYPE]->(s)
MERGE (p:Patient {Id: row.PATIENT})
MERGE (p)-[:HAS_ENCOUNTER]->(e)

// Add secondary label to Encounter nodes based on ENCOUNTERCLASS
CALL apoc.create.setLabels('Encounter', ['Encounter', row.ENCOUNTERCLASS], [e])
```

Explanation:
- The first section of the code creates SNOMED_CT nodes by loading the "encounters.csv" file and merging a new node for each unique 

In [38]:
prompt = f"""
Continuing the conversation, what Encounter-specific constraints should I create in the database? 
And what indexes should I create. New versions of Neo4j have replaced ASSERT with REQUIRE for 
creating constraints, so please update the code accordingly. 
"""

message_history, response_text = submit_query_and_append(prompt, message_history)

print(response_text)

When creating constraints in Neo4j, it is important to ensure that the constraints are unique and enforceable. Here are some suggestions for constraints and indexes based on the Encounter node and the data being loaded:

Constraints:
- :Encounter(Id) REQUIRE: This constraint ensures that each Encounter node has a unique ID property.

Indexes:
- :Encounter(Id): This index can help optimize queries that involve specific Encounter IDs.

To create these constraints and indexes, you can use the following Cypher code:

```
// Create constraints
CREATE CONSTRAINT ON (e:Encounter) ASSERT e.Id IS UNIQUE

// Create indexes
CREATE INDEX ON :Encounter(Id)
```

Note that the ASSERT keyword has been replaced with REQUIRE in newer versions of Neo4j, as you mentioned. The REQUIRE keyword works in the same way as ASSERT, but with a slightly different syntax.


## Create Encounter Index and Constraints

In [39]:
gds.run_cypher('''CREATE INDEX encounter_id IF NOT EXISTS FOR (n:Encounter) ON n.id''')
gds.run_cypher('''CREATE CONSTRAINT snomed_id IF NOT EXISTS FOR (n:SNOMED_CT) REQUIRE n.code IS UNIQUE''')

""


## Import Encounter Data and Relationships
The code ChatGPT provided is pretty good, and seems to be a roughly 80% solution, but it does not capture everything in the code provided in Graph Data Processing with Cypher. Therefore we will stick with the code provided in Ravi's book for our data load. 

In [40]:
gds.run_cypher('''
CYPHER runtime=slotted

UNWIND $encounters_data AS row
CALL {
WITH row
MERGE(e:Encounter {id: row.Id})
SET
    e.date=datetime(row.START),
    e.description=row.DESCRIPTION,
    e.totalCost = row.TOTAL_CLAIM_COST,
    e.isEnd = false

FOREACH (ignore in CASE WHEN row.STOP IS NOT NULL AND row.STOP <> '' THEN [1] ELSE [] END |
         SET e.end=datetime(row.STOP)
    )
FOREACH (ignore in CASE WHEN row.CODE IS NOT NULL AND row.CODE <> '' THEN [1] ELSE [] END |
         MERGE(s:SNOMED_CT {code:row.CODE})
         MERGE(e)-[:OF_TYPE]->(s)
    )
WITH row,e
CALL apoc.create.setLabels( e, [ 'Encounter', toUpper(left(row.ENCOUNTERCLASS, 1)) + right(row.ENCOUNTERCLASS, size(row.ENCOUNTERCLASS) - 1) ] ) 
YIELD node
    
WITH row,e
MERGE(p:Patient {id: row.PATIENT})
MERGE (p)-[:HAS_ENCOUNTER]->(e)

} IN TRANSACTIONS OF 1000 ROWS
''', {'encounters_data': encounters_df.to_dict('records')})

""


# Load Procedures Data into Neo4j

## Ask ChatGPT Specific Questions

In [41]:
procedures_df.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,REASONCODE,REASONDESCRIPTION
0,2013-07-24T11:27:37Z,2013-07-24T11:57:37Z,faf277d8-c5c7-406c-f139-9143c35ff4e8,eebc17d4-d133-24f2-3502-a3640d3feaba,268425006,Pelvis X-ray,431.40,,
1,2013-07-24T11:27:37Z,2013-07-24T12:53:37Z,faf277d8-c5c7-406c-f139-9143c35ff4e8,eebc17d4-d133-24f2-3502-a3640d3feaba,305428000,Admission to orthopedic department,431.40,359817006.0,Closed fracture of hip
2,2013-05-10T11:35:37Z,2013-05-10T11:46:01Z,75bef938-819a-ce94-8dc9-17dc50077a86,b4b95687-00df-e1e6-60ec-503775b3139d,76601001,Intramuscular injection,3186.39,,
3,2013-08-09T11:35:37Z,2013-08-09T11:47:00Z,75bef938-819a-ce94-8dc9-17dc50077a86,ddaa6f93-2b5f-d549-90f3-c05323396ef6,76601001,Intramuscular injection,3012.75,,
4,2013-11-08T11:35:37Z,2013-11-08T11:46:18Z,75bef938-819a-ce94-8dc9-17dc50077a86,6569b92d-1c2b-2a2f-b4bc-797f8aedaf88,76601001,Intramuscular injection,4077.14,,


In [42]:
# drop the first chats due to token limits
message_history = message_history[5:]

In [43]:
prompt = f"""
Continuing the conversation. I am going to load the procedure data into Neo4j from 
the procedures_df dataframe I shared earlier. I would like to use the following steps
for each row of the procedures_df dataframe:\n
####
1. Match to an existing Patient node using the Id property and the "PATIENT" feature in procedures_df.
2. Match the SNOMED_CT matching its 'code' property to the "CODE" feature in procedures_df. 
Create a new SNOMED_CT node with the 'code' property if one does not exist for this code. 
3. Set a 'description' property on the SNOMED_CT node using the 'Description' feature in procedures_df.
4. Set a secondary "Procedure" label on the SNOMED_CT code to identify it as a procedure.
5. Match the Encounter node on its 'id' property with the "ENCOUNTER" feature of procedures_df.
If the Encounter node does not exist, create a new Encounter node with "ENCOUNTER" as the id 
and a 'date' as the "START" feature. 
6. Match the Patient node with the Encounter node using a HAS_ENCOUNTER relationship.
7. Match that Encounter node with the Procedure node using a HAS_PROCEDURE relationship.
####
\n

Please provide me with Cypher code to load this data into Neo4j. 
Please also provide an explanation for the code to help me understand it.
"""

message_history, response_text = submit_query_and_append(prompt, message_history)

print(response_text)

Sure, here is the Cypher code to load the procedure data into Neo4j:

```
// Create or update SNOMED_CT nodes and add Procedure label
LOAD CSV WITH HEADERS FROM 'file:///procedures.csv' AS row
MERGE (s:SNOMED_CT {code: row.CODE})
ON CREATE SET s.description = row.Description
SET s:Procedure

// Create or update Encounter nodes and connect to Patient nodes
LOAD CSV WITH HEADERS FROM 'file:///procedures.csv' AS row
MERGE (e:Encounter {Id: row.ENCOUNTER})
ON CREATE SET e.START = row.START
MERGE (p:Patient {Id: row.PATIENT})
MERGE (p)-[:HAS_ENCOUNTER]->(e)

// Connect Encounter nodes to Procedure nodes
LOAD CSV WITH HEADERS FROM 'file:///procedures.csv' AS row
MATCH (e:Encounter {Id: row.ENCOUNTER})
MATCH (s:SNOMED_CT {code: row.CODE})
MERGE (e)-[:HAS_PROCEDURE]->(s)
```

Explanation:
- The first section of the code creates or updates SNOMED_CT nodes by loading the "procedures.csv" file and merging a new node for each unique SNOMED_CT code found in the file. If a node already exists for a 

In [44]:
prompt = f"""
Continuing the conversation, the code you provided is a good start but does not seem 
to follow the instructions. Let me clarify -- for each row in the dataframe, complete the 
seven steps I listed in order. I want to match and create nodes only when required, as I 
process each individual row in the procedure_df data.

Please provide updated Cypher code to perform these steps as defined.
"""

message_history, response_text = submit_query_and_append(prompt, message_history)

print(response_text)

Apologies for the confusion. Here is the updated Cypher code that follows the steps you provided for each row in the procedures_df dataframe:

```
// For each row in procedures_df, match or create Patient node
LOAD CSV WITH HEADERS FROM 'file:///procedures.csv' AS row
MERGE (p:Patient {Id: row.PATIENT})

// For each row in procedures_df, match or create Encounter node and connect to Patient node
LOAD CSV WITH HEADERS FROM 'file:///procedures.csv' AS row
MATCH (p:Patient {Id: row.PATIENT})
MERGE (e:Encounter {Id: row.ENCOUNTER})
ON CREATE SET e.START = row.START
MERGE (p)-[:HAS_ENCOUNTER]->(e)

// For each row in procedures_df, match or create SNOMED_CT node and connect to Encounter node
LOAD CSV WITH HEADERS FROM 'file:///procedures.csv' AS row
MATCH (e:Encounter {Id: row.ENCOUNTER})
MERGE (s:SNOMED_CT {code: row.CODE})
ON CREATE SET s.description = row.Description, s:Procedure
MERGE (e)-[:HAS_PROCEDURE]->(s)
```

Explanation:
- The first section of the code matches or creates a Patien

## Import Procedures Data and Relationships

As with before, the Cypher is an okay start, but still has limitations and is not what we would be able to use for our actual data loading. As I am creating this notebook, I am seeing it is likely a combination of the prompts (and clarifications) along with limits with ChatGPT. 

As with before, we will stick to the code provided in Graph Data Processing with Cypher. 

In [45]:
gds.run_cypher('''
CYPHER runtime=slotted

UNWIND $procedures_data AS row
CALL {
WITH row
MATCH (p:Patient {id:row.PATIENT})
MERGE (c:SNOMED_CT {code:row.CODE})
    SET c.description=row.DESCRIPTION, c:Procedure

MERGE (cs:Encounter {id:row.ENCOUNTER, isEnd: false})
  ON CREATE
  SET cs.date=datetime(row.START)

MERGE (p)-[:HAS_ENCOUNTER]->(cs)
MERGE (cs)-[:HAS_PROCEDURE]->(c)
} IN TRANSACTIONS OF 1000 ROWS
''', {'procedures_data': procedures_df.to_dict('records')})

""


# Load Medications Data into Neo4j

## Ask ChatGPT Specific Questions

In [46]:
medications_df.head()

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,2013-07-24T12:53:37Z,2013-08-20T11:16:28Z,faf277d8-c5c7-406c-f139-9143c35ff4e8,df166300-5a78-3502-a46a-832842197811,eebc17d4-d133-24f2-3502-a3640d3feaba,861467,Meperidine Hydrochloride 50 MG Oral Tablet,53.88,0.00,1,53.88,,
1,2013-07-24T12:53:37Z,2013-10-01T12:53:37Z,faf277d8-c5c7-406c-f139-9143c35ff4e8,df166300-5a78-3502-a46a-832842197811,eebc17d4-d133-24f2-3502-a3640d3feaba,313782,Acetaminophen 325 MG Oral Tablet,124.87,0.00,2,249.74,,
2,2014-10-05T11:54:48Z,2015-09-30T11:54:48Z,75bef938-819a-ce94-8dc9-17dc50077a86,0133f751-9229-3cfd-815f-b6d4979bdd6a,90b93931-eec7-ff2b-f759-0381562bc7fa,831533,Errin 28 Day Pack,441.21,411.21,12,5294.52,,
3,2014-10-31T11:35:37Z,2014-11-21T11:35:37Z,75bef938-819a-ce94-8dc9-17dc50077a86,0133f751-9229-3cfd-815f-b6d4979bdd6a,c6fcb2f9-a4b5-fb21-2db3-36ae3114cbee,310965,Ibuprofen 200 MG Oral Tablet,90.68,60.68,1,90.68,,
4,2015-09-30T11:54:48Z,2016-04-01T11:35:37Z,75bef938-819a-ce94-8dc9-17dc50077a86,0133f751-9229-3cfd-815f-b6d4979bdd6a,f90f36bc-72a9-8458-839a-68c3ef53695a,831533,Errin 28 Day Pack,680.38,650.38,6,4082.28,,


In [50]:
# drop the first chats due to token limits
message_history = message_history[5:]

In [51]:
prompt = f"""
Continuing the conversation. I am going to load the medications data into Neo4j from 
the medications_df dataframe I shared earlier. All of the steps will reference the medications_df 
dataframe or nodes and relationships already in the Neo4j database. I want to follow these steps 
for each row in the dataframe.
\n
####
1. Match to an existing Patient node using its 'id' property with the PATIENT feature in 
the medications_df dataframe. If the Patient node does not exist create a new Patient node 
with an 'id' property equal to the PATIENT feature of the medications_df dataframe.
2. Match to an existing "Drug" node based upon the "CODE" feature in medications_df. If it does not exist
create a new Drug node and set a 'code' property equal to the "CODE" feature, the 'description' property
equal to the "DESCRIPTION" feature, and the 'cost' property equal to the "BASE_COST" feature all from medications_df. 
3. Match to an existing Encounter node on its 'id' property and the "ENCOUNTER" feature in the dataframe.
If a matching Encounter node does not exist, creat it and set the 'id' property as the "ENCOUNTER" feature
from medications_df.
4. Connect the previously matched Patient node to the previously matched Encounter node with a HAS_ENCOUNTER relationship.
5. Connect the previously matched Encounter node to the previously matched Drug node with a HAS_DRUG relationship. 
6. If the 'REASONDESCRIPTION' feature in the medications_df row is not empty or null, match it to a SNOMED_CT node 
using the 'code' property. Then create a HAS_DIAGNOSIS relationship from the previously matched Encounter node 
to this new SNOMED_CT node. If a matching SNOMED_CT node does not exist, create a new one with 
the "CODE" feature of medications_df as the property 'code'.
7. If the 'STOP' feature in the curent row of medications_df is not empty or null, create a new Encounter 
node with the "ENCOUNTER" feature as the node property 'id, the "STOP" feature as the node property 'datetime', 
and the node property isEnd as a boolean of True. 
8. Connect the previously matched Patient node to the new Encounter node via a HAS_ENCOUNTER relationship.
9. Connect the Drug node to the new Encounter via a HAS_DRUG relationship.
10. Connect the first Encounter node to second Encounter node via a HAS_END relationship.

####
\n

Please provide me with Cypher code to load this data into Neo4j. 
Please also provide an explanation for the code to help me understand it.
"""

message_history, response_text = submit_query_and_append(prompt, message_history)

print(response_text)

Sure, here is the Cypher code to load the medication data into Neo4j:

```
// For each row in medications_df, match or create Patient node
UNWIND $medications_df AS row
MERGE (p:Patient {Id: row.PATIENT})

// For each row in medications_df, match or create Drug node
UNWIND $medications_df AS row
MERGE (d:Drug {code: row.CODE})
ON CREATE SET d.description = row.DESCRIPTION, d.cost = row.BASE_COST, d:Drug

// For each row in medications_df, match or create Encounter node and connect to Patient node
UNWIND $medications_df AS row
MERGE (p:Patient {Id: row.PATIENT})
MERGE (e:Encounter {Id: row.ENCOUNTER})
ON CREATE SET e.START = row.START
MERGE (p)-[:HAS_ENCOUNTER]->(e)

// For each row in medications_df, connect Encounter node to Drug node
UNWIND $medications_df AS row
MATCH (e:Encounter {Id: row.ENCOUNTER})
MATCH (d:Drug {code: row.CODE})
MERGE (e)-[:HAS_DRUG]->(d)

// For each row in medications_df, match or create SNOMED_CT node and connect to Encounter node
UNWIND $medications_df AS ro

## Import Medications Data and Relationships
The instructions to ingest the medications data were the most extensive in this notebook, and it is no surprise that ChatGPT seemed to struggle with them. It is worth noting that I went through several versions of the instructions above when creating the notebook. However, as before, I will stick with the code provided in Graph Data Processing with Cypher.

In [52]:
gds.run_cypher('''
CYPHER runtime=slotted

UNWIND $medications_data AS row
CALL {
WITH row
MERGE (p:Patient {id: row.PATIENT})
MERGE (d:Drug {code: row.CODE})
    SET d.description = row.DESCRIPTION,
        d.cost = toFloat(row.BASE_COST)
MERGE (ps:Encounter {id: row.ENCOUNTER, isEnd: false})
MERGE (ps)-[:HAS_DRUG]->(d)
MERGE (p)-[:HAS_ENCOUNTER]->(ps)

FOREACH (ignore in CASE WHEN 
                        row.REASONCODE IS NOT NULL AND
                        row.REASONCODE <> '' THEN [1] ELSE [] END | 
        MERGE (s:SNOMED_CT {code: row.CODE})
        SET s:Diagnosis, s.description = row.REASONDESCRIPTION
        MERGE (ps)-[:HAS_DIAGNOSIS]->(s)
)

WITH row, ps, p
  WHERE row.STOP IS NOT NULL and row.STOP <> ''
CREATE (pe:Encounter {id:row.ENCOUNTER, date:datetime(row.STOP)})
SET pe.isEnd=true
CREATE (p)-[:HAS_ENCOUNTER]->(pe)
CREATE (pe)-[:HAS_DRUG]->(d)
CREATE (ps)-[:HAS_END]->(pe)
} IN TRANSACTIONS OF 1000 ROWS
''', {'medications_data': medications_df.to_dict('records')})

""


In [54]:
prompt = """
I want you to act as an experienced graph data scientist who works at Neo4j. 
Please explain the following Cypher code that I used to load the Procedures data 
into the Neo4j database:

####
CYPHER runtime=slotted

UNWIND $medications_data AS row
CALL {
WITH row
MERGE (p:Patient {id: row.PATIENT})
MERGE (d:Drug {code: row.CODE})
    SET d.description = row.DESCRIPTION,
        d.cost = toFloat(row.BASE_COST)
MERGE (ps:Encounter {id: row.ENCOUNTER, isEnd: false})
MERGE (ps)-[:HAS_DRUG]->(d)
MERGE (p)-[:HAS_ENCOUNTER]->(ps)

FOREACH (ignore in CASE WHEN 
                        row.REASONCODE IS NOT NULL AND
                        row.REASONCODE <> '' THEN [1] ELSE [] END | 
        MERGE (s:SNOMED_CT {code: row.CODE})
        SET s:Diagnosis, s.description = row.REASONDESCRIPTION
        MERGE (ps)-[:HAS_DIAGNOSIS]->(s)
)

WITH row, ps, p
  WHERE row.STOP IS NOT NULL and row.STOP <> ''
CREATE (pe:Encounter {id:row.ENCOUNTER, date:datetime(row.STOP)})
SET pe.isEnd=true
CREATE (p)-[:HAS_ENCOUNTER]->(pe)
CREATE (pe)-[:HAS_DRUG]->(d)
CREATE (ps)-[:HAS_END]->(pe)
} IN TRANSACTIONS OF 1000 ROWS
####

"""

message_history, response_text = submit_query_and_append(prompt, message_history)

print(response_text)

I apologize, but the Cypher code you provided is actually for loading medication data, not procedures data. However, I can still provide an explanation of the code for loading medication data:

- The `UNWIND` clause is used to iterate over each row in the `$medications_data` parameter.
- The `CALL` clause is used to execute a subquery that creates or updates the Drug nodes and their relationships to Patient and Encounter nodes.
- The `WITH` clause is used to pass the current row to the next part of the query.
- The `MERGE` clause is used to create or update the Patient node with the corresponding ID using the `id` property.
- The `MERGE` clause is used to create or update the Drug node with the corresponding code using the `code` property. The `SET` clause is used to set the `description` and `cost` properties of the Drug node using the corresponding values from the `$medications_data` parameter.
- The `MERGE` clause is used to create or update the Encounter node with the corresponding

In [ ]:
print('data load complete!')

data load complete!
